![glassdoor](https://github.com/jasonchanhku/DataScienceDemand/blob/master/images/glassdoor.png?raw=true)

# Data Science Demand in Hong Kong
### by Jason Chan Jin An

# Introduction

This project aims to gauge the data science job demands in Hong Kong
in the past 30 days rolling based on job posts from Glassdoor. Glassdoor was 
the preferred data source because its wide array of available information:

* Job Title
* Company Name
* Link
* Company Rating
* Job Description
* Company Size
* Year Founded
* Company Type
* Industry
* Company Revenue
* CEO (sentiment)
* Recommend Percentage
* Approval Percentage

# Questions to be Answered

Exploratory Data Analysis (EDA) is performed in this notebook from a statistical and data standpoint and seeks to answer the following questions: 
* Who are hiring data scientists in Hong Kong ?
    * Big / small companies ?
    * What kind of industries ?
    * Good company feedback and approval ?
* Do company ratings differ from company types / industry / etc ?
* Given my preference of company type, which jobs suits me most ?

## Dataset

The dataset was obtained by building a python web scraper using `selenium` and the script is saved as `scraper.py` in the repository.

***

# Data Prep and Libraries

In [287]:
# Libraries used
import pandas as pd
import numpy as np

In [288]:
# Data prep
df = pd.read_csv('https://raw.githubusercontent.com/jasonchanhku/DataScienceDemand/master/data/glassdoor_data.csv')
df.head(5)

,Title,Company,Link,Rating,Job_Description,Size,Founded,Company_Type,Industry,Revenue,CEO,Recommend,Approve
0,"CIB QR - Risk Quantitative Research, Equity De...",J.P. Morgan,https://www.glassdoor.com/partner/jobListing.h...,3.7,J.P. Morgans Corporate & Investment Bank is a ...,10000+ employees,1799,Public (JPM),Finance,$10+ billion (USD) per year,Jamie Dimon,76.0,93.0
1,Quantitative Research - M/F VIE,Societe Generale,https://www.glassdoor.com/partner/jobListing.h...,3.4,Environment\n\nYour environment\nSG CIB is the...,10000+ employees,1864,Public (GLE),Finance,$10+ billion (USD) per year,Frederic Oudea,68.0,83.0
2,Data Analyst - Modeling,Transunion,https://www.glassdoor.com/partner/jobListing.h...,3.9,Dynamics of the Role\n\nThe incumbent\nis expe...,1001 to 5000 employees,1968,Public (TRU),Finance,$1 to $2 billion (USD) per year,Jim Peck,75.0,93.0
3,Data Scientist,Lenovo,https://www.glassdoor.com/partner/jobListing.h...,3.3,Position Description\nDesign data mining and m...,10000+ employees,1984,Public (LNVGY),Information Technology,$10+ billion (USD) per year,Yang Yuanqing,57.0,64.0
4,Quantitative Researcher,Societe Generale,https://www.glassdoor.com/partner/jobListing.h...,3.4,Environment\n\nSG CIB is the Corporate and Inv...,10000+ employees,1864,Public (GLE),Finance,$10+ billion (USD) per year,Frederic Oudea,68.0,83.0


In [289]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 13 columns):
Title              211 non-null object
Company            211 non-null object
Link               211 non-null object
Rating             189 non-null float64
Job_Description    211 non-null object
Size               207 non-null object
Founded            207 non-null object
Company_Type       207 non-null object
Industry           207 non-null object
Revenue            207 non-null object
CEO                190 non-null object
Recommend          190 non-null float64
Approve            190 non-null float64
dtypes: float64(3), object(10)
memory usage: 21.5+ KB


# Data Preprocessing

## Exclude the following rows:
* Rows of comapny that is 'RegionUP' as they have been flagged as a scam company
* Drop rows with NA columns since insignificnat, will make it easier for machine learning
* Drop rows without CEO approval ratings since very small amount of them do not have

## Basic cleaning of Job_Description column:
* Strip leading and trailing whitespaces
* Strip '\n' from the job column description and other html tags
* Replace double space with single space

## Improve readability of revenue column
* Remove unecessary parts of text
* Impute values of 'Unknown / Not Applicable' using employee size
* Preserve ordinality by mapping

## Impute Size column
* Impute using revenue column

## Company_Type column
* Remove stock tickers

### Excluding Rows

In [290]:
# Exclude outlier RegionUP as it is a scam company
df = df[df['Company'] != 'RegionUP']

# Remove NAs
df = df.dropna()

# Drop those without Approval scores
df = df[df['Approve'] != -1]

# Remove unknown size companies
df = df[df['Size'] != 'Unknown']

df.shape

(140, 13)

In [291]:
# Cleanse job description column
def jd_cleanse(col):
    
    col = col.apply(lambda x: x.strip())
    col = col.apply(lambda x: x.replace('\n', ' '))
    col = col.apply(lambda x: x.replace('  ', ' '))
    
    return col

# Cleanse the revenue column

def rev_cleanse(col):
    
    col = col.apply(lambda x: x.replace('per year', ''))
    col = col.apply(lambda x: x.replace('(USD)', '')) 
    col = col.apply(lambda x: x.strip())
    col = col.replace('Unknown / Non-Applicable', None)
    mapping = pd.pivot_table(df[['Size', 'Revenue']].drop_duplicates(),index=['Size'], columns=['Revenue'], aggfunc=np.count_nonzero).idxmax(axis =1)
    col.fillna(df.Size.map(mapping))
    
    return col

### Job_Description Column

In [292]:
df['Job_Description'] = jd_cleanse(df['Job_Description'])

In [293]:
df['Job_Description'].head()

0    J.P. Morgans Corporate & Investment Bank is a ...
1    Environment Your environment SG CIB is the Cor...
2    Dynamics of the Role The incumbent is expected...
3    Position Description Design data mining and ma...
4    Environment SG CIB is the Corporate and Invest...
Name: Job_Description, dtype: object

### Revenue Column

How many are Unknown / Not Applicable ?
* Around 21 unknown revenues

In [294]:
df['Revenue'].value_counts()

$10+ billion (USD) per year                  57
Unknown / Non-Applicable                     20
$5 to $10 billion (USD) per year             18
$2 to $5 billion (USD) per year               8
$1 to $2 billion (USD) per year               8
$50 to $100 million (USD) per year            7
$100 to $500 million (USD) per year           7
$10 to $25 million (USD) per year             4
$500 million to $1 billion (USD) per year     4
Less than $1 million (USD) per year           2
$25 to $50 million (USD) per year             2
$1 to $5 million (USD) per year               2
Unknown / Non-Applicable per year             1
Name: Revenue, dtype: int64

#### Imputing Revenue by most common occrence

* Impute revenue based on most common occurence grouped by company size


### Pivot Table of Revenue mode occurence for each Size

In [295]:
pd.pivot_table(df[['Size', 'Revenue']].drop_duplicates(),index=['Size'], columns=['Revenue'], aggfunc=np.count_nonzero).idxmax(axis =1)

Size
1 to 50 employees              $1 to $5 million (USD) per year
10000+ employees                   $10+ billion (USD) per year
1001 to 5000 employees         $1 to $2 billion (USD) per year
201 to 500 employees           $1 to $5 million (USD) per year
5001 to 10000 employees        $1 to $2 billion (USD) per year
501 to 1000 employees      $100 to $500 million (USD) per year
51 to 200 employees          $10 to $25 million (USD) per year
dtype: object

In [296]:
dict(mapping)

{'1 to 50 employees': '$1 to $5 million',
 '10000+ employees': '$10+ billion',
 '1001 to 5000 employees': '$1 to $2 billion',
 '201 to 500 employees': '$1 to $5 million',
 '5001 to 10000 employees': '$1 to $2 billion',
 '501 to 1000 employees': '$100 to $500 million',
 '51 to 200 employees': '$10 to $25 million'}

In [297]:
df['Revenue'] = rev_cleanse(df['Revenue'])

In [299]:
df['Revenue'].isnull().sum()

0